# Trojan Binary Analysis

**Goal**: Analyze the Jupiter Trojan populations from the migration simulation to identify potential binary systems in L4 and L5 regions.

We'll look for:
1. Close pairs of planetesimals (potential binaries)
2. Their relative orbital properties (eccentricity, separation)
3. Compare L4 vs L5 binary populations

## 1. Setup and Load Data

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pickle
from scipy.spatial.distance import pdist, squareform

print('Libraries loaded successfully!')

Libraries loaded successfully!


In [5]:
# Load the simulation data
filename = 'jupiter_migration_5000kyr_inward.pkl'

print(f'Loading data from {filename}...')
with open(filename, 'rb') as f:
    data = pickle.load(f)

print('='*60)
print('Data loaded successfully!')
print('='*60)
print(f"Available keys: {list(data.keys())}")

# Extract data
times = data['times']
snapshots_massive = data['snapshots_massive']
snapshots_planetesimals = data['snapshots_planetesimals']

print(f"\nSimulation info:")
print(f"  Time snapshots: {len(times)}")
print(f"  Time range: {times[0]:.1f} - {times[-1]:.1f} years")
print(f"  Massive bodies: {len(snapshots_massive[0])}")
print(f"  Initial planetesimals: {len(snapshots_planetesimals[0])}")
print(f"  Final planetesimals: {len(snapshots_planetesimals[-1])}")

# Check if L4/L5 counts are already stored
if 'l4_counts' in data and 'l5_counts' in data:
    l4_counts = data['l4_counts']
    l5_counts = data['l5_counts']
    print(f"\nFinal Trojan populations:")
    print(f"  L4: {l4_counts[-1]}")
    print(f"  L5: {l5_counts[-1]}")
else:
    print("\nNote: L4/L5 counts not pre-computed, will identify Trojans from scratch")
print('='*60)

Loading data from jupiter_migration_5000kyr_inward.pkl...
Data loaded successfully!
Available keys: ['times', 'snapshots_massive', 'snapshots_planetesimals', 'parameters']

Simulation info:
  Time snapshots: 201
  Time range: 0.0 - 500000.1 years
  Massive bodies: 8
  Initial planetesimals: 10000
  Final planetesimals: 10000

Note: L4/L5 counts not pre-computed, will identify Trojans from scratch


## 2. Identify Trojan Populations (L4 and L5)

In [7]:
def identify_trojans(massive_bodies, planetesimals):
    """Identify planetesimals in Jupiter's L4 and L5 Trojan regions.
    
    Returns:
    --------
    l4_indices, l5_indices : arrays
        Indices of particles in L4 and L5 regions
    """
    jupiter = massive_bodies[4]  # Jupiter at index 4
    
    # Jupiter's angle and semi-major axis (extract values from AMUSE units)
    jupiter_x = jupiter.x.value_in(jupiter.x.unit)
    jupiter_y = jupiter.y.value_in(jupiter.y.unit)
    jupiter_z = jupiter.z.value_in(jupiter.z.unit)
    
    lambda_j = np.arctan2(jupiter_y, jupiter_x)
    a_j = np.sqrt(jupiter_x**2 + jupiter_y**2 + jupiter_z**2)
    
    l4_indices = []
    l5_indices = []
    
    for i, p in enumerate(planetesimals):
        # Particle position and angle (extract values)
        p_x = p.x.value_in(p.x.unit)
        p_y = p.y.value_in(p.y.unit)
        p_z = p.z.value_in(p.z.unit)
        
        lam = np.arctan2(p_y, p_x)
        r = np.sqrt(p_x**2 + p_y**2 + p_z**2)
        
        # Angular separation from Jupiter (wrapped to [-pi, pi])
        dlam = np.arctan2(np.sin(lam - lambda_j), np.cos(lam - lambda_j))
        
        # Co-orbital criterion: within 7% of Jupiter's semi-major axis
        is_coorb = abs(r / a_j - 1) < 0.07
        
        if is_coorb:
            # L4: +60° from Jupiter (±25° sector)
            if np.deg2rad(60-25) < dlam < np.deg2rad(60+25):
                l4_indices.append(i)
            # L5: -60° from Jupiter (±25° sector)
            elif np.deg2rad(-60-25) < dlam < np.deg2rad(-60+25):
                l5_indices.append(i)
    
    return np.array(l4_indices), np.array(l5_indices)

# Identify Trojans in the final snapshot
final_massive = snapshots_massive[-1]
final_planetesimals = snapshots_planetesimals[-1]

l4_idx, l5_idx = identify_trojans(final_massive, final_planetesimals)

print('='*60)
print('Trojan Identification (Final Snapshot)')
print('='*60)
print(f'Total planetesimals: {len(final_planetesimals)}')
print(f'L4 Trojans: {len(l4_idx)}')
print(f'L5 Trojans: {len(l5_idx)}')
print(f'Total Trojans: {len(l4_idx) + len(l5_idx)}')
print(f'Trojan fraction: {100*(len(l4_idx) + len(l5_idx))/len(final_planetesimals):.1f}%')
print('='*60)

Trojan Identification (Final Snapshot)
Total planetesimals: 10000
L4 Trojans: 77
L5 Trojans: 74
Total Trojans: 151
Trojan fraction: 1.5%


## 3. Binary Detection Algorithm

We'll identify binaries using physical criteria:
- **Mutual Hill radius**: Particles within ~3 mutual Hill radii are gravitationally bound
- **Relative velocity**: Check if they're in bound orbits around each other
- **Separation**: Typical asteroid binaries have separations of 2-10 Hill radii

In [10]:
def find_binaries(particles, sun_mass=1.0, max_hill_radii=5.0):
    """Find potential binary pairs among particles.
    
    Parameters:
    -----------
    particles : AMUSE particle set or list
        Particles to search for binaries
    sun_mass : float
        Mass of the Sun (solar masses)
    max_hill_radii : float
        Maximum separation in mutual Hill radii to consider as binary
    
    Returns:
    --------
    binaries : list of dict
        Each dict contains: 'i', 'j', 'separation', 'hill_radii', 
        'rel_velocity', 'eccentricity', 'bound'
    """
    n = len(particles)
    if n < 2:
        return []
    
    # Extract positions and velocities (handling AMUSE units)
    positions = []
    velocities = []
    for p in particles:
        # Extract numerical values from AMUSE units
        if hasattr(p.x, 'value_in'):
            # AMUSE particle with units - extract to float
            pos = [float(p.x.value_in(p.x.unit)), 
                   float(p.y.value_in(p.y.unit)), 
                   float(p.z.value_in(p.z.unit))]
            vel = [float(p.vx.value_in(p.vx.unit)), 
                   float(p.vy.value_in(p.vy.unit)), 
                   float(p.vz.value_in(p.vz.unit))]
        else:
            # Plain numbers
            pos = [float(p.x), float(p.y), float(p.z)]
            vel = [float(p.vx), float(p.vy), float(p.vz)]
        positions.append(pos)
        velocities.append(vel)
    
    # Ensure float64 dtype for scipy
    positions = np.array(positions, dtype=np.float64)
    velocities = np.array(velocities, dtype=np.float64)
    
    # Compute all pairwise distances
    dist_matrix = squareform(pdist(positions))
    
    binaries = []
    checked = set()
    
    for i in range(n):
        for j in range(i+1, n):
            if (i, j) in checked:
                continue
            
            # Physical separation
            r_ij = dist_matrix[i, j]
            
            # Semi-major axes (distance from Sun)
            r_i = np.linalg.norm(positions[i])
            r_j = np.linalg.norm(positions[j])
            a_avg = (r_i + r_j) / 2.0
            
            # Mutual Hill radius (assuming equal mass planetesimals)
            # R_H = a * (m1 + m2 / 3*M_sun)^(1/3)
            # For small mass planetesimals, approximate as:
            # R_H ≈ a * (2m / 3*M_sun)^(1/3)
            # Assume planetesimal mass ~ 10^-10 MSun (rough estimate)
            m_planetes = 1e-10  # Very rough estimate
            R_hill = a_avg * (2 * m_planetes / (3 * sun_mass))**(1/3)
            
            # Separation in Hill radii
            sep_hill = r_ij / R_hill
            
            if sep_hill < max_hill_radii:
                # Calculate relative velocity
                v_rel = velocities[j] - velocities[i]
                v_rel_mag = np.linalg.norm(v_rel)
                
                # Relative position
                r_rel = positions[j] - positions[i]
                
                # Mutual orbital elements (treating as 2-body problem)
                # Specific energy: E = 0.5*v^2 - G*m_total/r
                # For bound orbit: E < 0
                G = 39.478  # AU^3 / (MSun * yr^2)
                m_total = 2 * m_planetes
                E_specific = 0.5 * v_rel_mag**2 - G * m_total / r_ij
                
                is_bound = E_specific < 0
                
                # Eccentricity (if bound)
                if is_bound:
                    # Angular momentum
                    h = np.cross(r_rel, v_rel)
                    h_mag = np.linalg.norm(h)
                    
                    # Semi-major axis
                    a_mutual = -G * m_total / (2 * E_specific)
                    
                    # Eccentricity: e = sqrt(1 + 2*E*h^2/(G*m)^2)
                    e_squared = 1 + 2 * E_specific * h_mag**2 / (G * m_total)**2
                    e = np.sqrt(max(0, e_squared))
                else:
                    a_mutual = None
                    e = None
                
                binaries.append({
                    'i': i,
                    'j': j,
                    'separation_AU': r_ij,
                    'separation_hill': sep_hill,
                    'rel_velocity': v_rel_mag,
                    'eccentricity': e,
                    'semi_major_axis': a_mutual,
                    'bound': is_bound
                })
                
                checked.add((i, j))
    
    return binaries

print('Binary detection function defined!')

Binary detection function defined!


## 4. Search for Binaries in L4 and L5

In [11]:
# Extract L4 and L5 particle subsets
l4_particles = [final_planetesimals[i] for i in l4_idx]
l5_particles = [final_planetesimals[i] for i in l5_idx]

print('Searching for binaries in L4...')
l4_binaries = find_binaries(l4_particles, max_hill_radii=5.0)

print('Searching for binaries in L5...')
l5_binaries = find_binaries(l5_particles, max_hill_radii=5.0)

# Filter for bound binaries only
l4_bound = [b for b in l4_binaries if b['bound']]
l5_bound = [b for b in l5_binaries if b['bound']]

print('='*60)
print('Binary Search Results (Final Snapshot)')
print('='*60)
print(f'\nL4 Region ({len(l4_particles)} particles):')
print(f'  Total close pairs (< 5 R_Hill): {len(l4_binaries)}')
print(f'  Gravitationally bound pairs: {len(l4_bound)}')
if len(l4_bound) > 0:
    print(f'  Binary fraction: {100*len(l4_bound)/len(l4_particles):.2f}%')
    separations = [b['separation_hill'] for b in l4_bound]
    eccentricities = [b['eccentricity'] for b in l4_bound if b['eccentricity'] is not None]
    print(f'  Separation range: {min(separations):.2f} - {max(separations):.2f} R_Hill')
    if eccentricities:
        print(f'  Eccentricity range: {min(eccentricities):.3f} - {max(eccentricities):.3f}')

print(f'\nL5 Region ({len(l5_particles)} particles):')
print(f'  Total close pairs (< 5 R_Hill): {len(l5_binaries)}')
print(f'  Gravitationally bound pairs: {len(l5_bound)}')
if len(l5_bound) > 0:
    print(f'  Binary fraction: {100*len(l5_bound)/len(l5_particles):.2f}%')
    separations = [b['separation_hill'] for b in l5_bound]
    eccentricities = [b['eccentricity'] for b in l5_bound if b['eccentricity'] is not None]
    print(f'  Separation range: {min(separations):.2f} - {max(separations):.2f} R_Hill')
    if eccentricities:
        print(f'  Eccentricity range: {min(eccentricities):.3f} - {max(eccentricities):.3f}')

print('='*60)

Searching for binaries in L4...
Searching for binaries in L5...
Binary Search Results (Final Snapshot)

L4 Region (77 particles):
  Total close pairs (< 5 R_Hill): 0
  Gravitationally bound pairs: 0

L5 Region (74 particles):
  Total close pairs (< 5 R_Hill): 0
  Gravitationally bound pairs: 0


## 5. Visualize Binary Properties

In [12]:
if len(l4_bound) > 0 or len(l5_bound) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Plot 1: Separation distribution
    ax = axes[0, 0]
    if len(l4_bound) > 0:
        l4_seps = [b['separation_hill'] for b in l4_bound]
        ax.hist(l4_seps, bins=20, alpha=0.6, color='blue', label=f'L4 (n={len(l4_bound)})')
    if len(l5_bound) > 0:
        l5_seps = [b['separation_hill'] for b in l5_bound]
        ax.hist(l5_seps, bins=20, alpha=0.6, color='red', label=f'L5 (n={len(l5_bound)})')
    ax.set_xlabel('Separation (Hill Radii)', fontsize=12)
    ax.set_ylabel('Number of Binaries', fontsize=12)
    ax.set_title('Binary Separation Distribution', fontsize=14)
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Plot 2: Eccentricity distribution
    ax = axes[0, 1]
    if len(l4_bound) > 0:
        l4_ecc = [b['eccentricity'] for b in l4_bound if b['eccentricity'] is not None]
        if l4_ecc:
            ax.hist(l4_ecc, bins=20, range=(0, 1), alpha=0.6, color='blue', label=f'L4 (n={len(l4_ecc)})')
    if len(l5_bound) > 0:
        l5_ecc = [b['eccentricity'] for b in l5_bound if b['eccentricity'] is not None]
        if l5_ecc:
            ax.hist(l5_ecc, bins=20, range=(0, 1), alpha=0.6, color='red', label=f'L5 (n={len(l5_ecc)})')
    ax.set_xlabel('Eccentricity', fontsize=12)
    ax.set_ylabel('Number of Binaries', fontsize=12)
    ax.set_title('Binary Eccentricity Distribution', fontsize=14)
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Plot 3: Separation vs Eccentricity
    ax = axes[1, 0]
    if len(l4_bound) > 0:
        l4_seps = [b['separation_hill'] for b in l4_bound if b['eccentricity'] is not None]
        l4_ecc = [b['eccentricity'] for b in l4_bound if b['eccentricity'] is not None]
        if l4_seps:
            ax.scatter(l4_seps, l4_ecc, alpha=0.6, s=50, color='blue', label='L4')
    if len(l5_bound) > 0:
        l5_seps = [b['separation_hill'] for b in l5_bound if b['eccentricity'] is not None]
        l5_ecc = [b['eccentricity'] for b in l5_bound if b['eccentricity'] is not None]
        if l5_seps:
            ax.scatter(l5_seps, l5_ecc, alpha=0.6, s=50, color='red', label='L5')
    ax.set_xlabel('Separation (Hill Radii)', fontsize=12)
    ax.set_ylabel('Eccentricity', fontsize=12)
    ax.set_title('Binary Separation vs Eccentricity', fontsize=14)
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Plot 4: Relative velocity distribution
    ax = axes[1, 1]
    if len(l4_bound) > 0:
        l4_vel = [b['rel_velocity'] for b in l4_bound]
        ax.hist(l4_vel, bins=20, alpha=0.6, color='blue', label=f'L4 (n={len(l4_bound)})')
    if len(l5_bound) > 0:
        l5_vel = [b['rel_velocity'] for b in l5_bound]
        ax.hist(l5_vel, bins=20, alpha=0.6, color='red', label=f'L5 (n={len(l5_bound)})')
    ax.set_xlabel('Relative Velocity (AU/yr)', fontsize=12)
    ax.set_ylabel('Number of Binaries', fontsize=12)
    ax.set_title('Binary Relative Velocity Distribution', fontsize=14)
    ax.legend()
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print('No bound binaries found to visualize.')

No bound binaries found to visualize.


## 6. Detailed Binary Pair Analysis

Let's look at individual binary pairs in detail.

In [ ]:
# Show top 10 closest bound binaries in each region
print('='*70)
print('TOP 10 CLOSEST BINARIES IN L4')
print('='*70)
if len(l4_bound) > 0:
    l4_sorted = sorted(l4_bound, key=lambda x: x['separation_hill'])
    for idx, b in enumerate(l4_sorted[:10], 1):
        print(f"\nBinary {idx}:")
        print(f"  Pair indices: ({b['i']}, {b['j']})")
        print(f"  Separation: {b['separation_AU']:.6f} AU ({b['separation_hill']:.3f} R_Hill)")
        print(f"  Relative velocity: {b['rel_velocity']:.6f} AU/yr")
        if b['eccentricity'] is not None:
            print(f"  Eccentricity: {b['eccentricity']:.4f}")
        if b['semi_major_axis'] is not None:
            print(f"  Mutual semi-major axis: {b['semi_major_axis']:.6f} AU")
else:
    print("No bound binaries found in L4")

print('\n' + '='*70)
print('TOP 10 CLOSEST BINARIES IN L5')
print('='*70)
if len(l5_bound) > 0:
    l5_sorted = sorted(l5_bound, key=lambda x: x['separation_hill'])
    for idx, b in enumerate(l5_sorted[:10], 1):
        print(f"\nBinary {idx}:")
        print(f"  Pair indices: ({b['i']}, {b['j']})")
        print(f"  Separation: {b['separation_AU']:.6f} AU ({b['separation_hill']:.3f} R_Hill)")
        print(f"  Relative velocity: {b['rel_velocity']:.6f} AU/yr")
        if b['eccentricity'] is not None:
            print(f"  Eccentricity: {b['eccentricity']:.4f}")
        if b['semi_major_axis'] is not None:
            print(f"  Mutual semi-major axis: {b['semi_major_axis']:.6f} AU")
else:
    print("No bound binaries found in L5")

print('='*70)

## 7. Statistical Comparison: L4 vs L5 Binaries

In [ ]:
print('='*70)
print('STATISTICAL COMPARISON: L4 vs L5 BINARIES')
print('='*70)

if len(l4_bound) > 0:
    l4_seps = np.array([b['separation_hill'] for b in l4_bound])
    l4_ecc = np.array([b['eccentricity'] for b in l4_bound if b['eccentricity'] is not None])
    l4_vel = np.array([b['rel_velocity'] for b in l4_bound])
    
    print("\nL4 Statistics:")
    print(f"  Number of binaries: {len(l4_bound)}")
    print(f"  Binary fraction: {100*len(l4_bound)/len(l4_particles):.3f}%")
    print(f"  Separation (R_Hill):")
    print(f"    Mean: {l4_seps.mean():.3f} ± {l4_seps.std():.3f}")
    print(f"    Median: {np.median(l4_seps):.3f}")
    print(f"    Range: [{l4_seps.min():.3f}, {l4_seps.max():.3f}]")
    if len(l4_ecc) > 0:
        print(f"  Eccentricity:")
        print(f"    Mean: {l4_ecc.mean():.3f} ± {l4_ecc.std():.3f}")
        print(f"    Median: {np.median(l4_ecc):.3f}")
        print(f"    Range: [{l4_ecc.min():.3f}, {l4_ecc.max():.3f}]")
    print(f"  Relative velocity (AU/yr):")
    print(f"    Mean: {l4_vel.mean():.6f} ± {l4_vel.std():.6f}")
    print(f"    Median: {np.median(l4_vel):.6f}")
else:
    print("\nL4: No binaries found")

if len(l5_bound) > 0:
    l5_seps = np.array([b['separation_hill'] for b in l5_bound])
    l5_ecc = np.array([b['eccentricity'] for b in l5_bound if b['eccentricity'] is not None])
    l5_vel = np.array([b['rel_velocity'] for b in l5_bound])
    
    print("\nL5 Statistics:")
    print(f"  Number of binaries: {len(l5_bound)}")
    print(f"  Binary fraction: {100*len(l5_bound)/len(l5_particles):.3f}%")
    print(f"  Separation (R_Hill):")
    print(f"    Mean: {l5_seps.mean():.3f} ± {l5_seps.std():.3f}")
    print(f"    Median: {np.median(l5_seps):.3f}")
    print(f"    Range: [{l5_seps.min():.3f}, {l5_seps.max():.3f}]")
    if len(l5_ecc) > 0:
        print(f"  Eccentricity:")
        print(f"    Mean: {l5_ecc.mean():.3f} ± {l5_ecc.std():.3f}")
        print(f"    Median: {np.median(l5_ecc):.3f}")
        print(f"    Range: [{l5_ecc.min():.3f}, {l5_ecc.max():.3f}]")
    print(f"  Relative velocity (AU/yr):")
    print(f"    Mean: {l5_vel.mean():.6f} ± {l5_vel.std():.6f}")
    print(f"    Median: {np.median(l5_vel):.6f}")
else:
    print("\nL5: No binaries found")

# Comparison
if len(l4_bound) > 0 and len(l5_bound) > 0:
    print("\nComparison:")
    print(f"  L4/L5 binary ratio: {len(l4_bound)/len(l5_bound):.3f}")
    print(f"  L4 vs L5 mean separation: {l4_seps.mean():.3f} vs {l5_seps.mean():.3f} R_Hill")
    if len(l4_ecc) > 0 and len(l5_ecc) > 0:
        print(f"  L4 vs L5 mean eccentricity: {l4_ecc.mean():.3f} vs {l5_ecc.mean():.3f}")

print('='*70)

## 8. Scan All Simulation Files for Binaries

Search through all available .pkl files to find binary systems across different simulations.

In [13]:
import glob
import os

# Find all .pkl files in the current directory
pkl_files = glob.glob('*.pkl')
pkl_files.sort()

print('='*80)
print('BINARY SEARCH ACROSS ALL SIMULATION FILES')
print('='*80)
print(f'\nFound {len(pkl_files)} .pkl files:\n')

results_summary = []

for filename in pkl_files:
    print(f'\n{"="*80}')
    print(f'Analyzing: {filename}')
    print(f'{"="*80}')
    
    try:
        # Load file
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        # Check if it has the expected structure
        if 'snapshots_massive' not in data or 'snapshots_planetesimals' not in data:
            print(f'  ⚠️  Skipping - missing required data keys')
            continue
        
        times = data['times']
        snapshots_massive = data['snapshots_massive']
        snapshots_planetesimals = data['snapshots_planetesimals']
        
        # Use final snapshot
        final_massive = snapshots_massive[-1]
        final_planetesimals = snapshots_planetesimals[-1]
        
        print(f'  Time range: {times[0]:.1f} - {times[-1]:.1f} yr')
        print(f'  Final planetesimals: {len(final_planetesimals)}')
        
        # Identify Trojans
        l4_idx, l5_idx = identify_trojans(final_massive, final_planetesimals)
        print(f'  L4 Trojans: {len(l4_idx)}')
        print(f'  L5 Trojans: {len(l5_idx)}')
        
        if len(l4_idx) == 0 and len(l5_idx) == 0:
            print(f'  ℹ️  No Trojans found - skipping binary search')
            results_summary.append({
                'filename': filename,
                'total_particles': len(final_planetesimals),
                'l4_trojans': 0,
                'l5_trojans': 0,
                'l4_binaries': 0,
                'l5_binaries': 0,
                'status': 'No Trojans'
            })
            continue
        
        # Search for binaries in L4
        l4_binaries = []
        l4_bound = []
        if len(l4_idx) >= 2:
            l4_particles = [final_planetesimals[i] for i in l4_idx]
            print(f'  Searching L4 for binaries...')
            l4_binaries = find_binaries(l4_particles, max_hill_radii=5.0)
            l4_bound = [b for b in l4_binaries if b['bound']]
            print(f'    Close pairs: {len(l4_binaries)}, Bound: {len(l4_bound)}')
        
        # Search for binaries in L5
        l5_binaries = []
        l5_bound = []
        if len(l5_idx) >= 2:
            l5_particles = [final_planetesimals[i] for i in l5_idx]
            print(f'  Searching L5 for binaries...')
            l5_binaries = find_binaries(l5_particles, max_hill_radii=5.0)
            l5_bound = [b for b in l5_binaries if b['bound']]
            print(f'    Close pairs: {len(l5_binaries)}, Bound: {len(l5_bound)}')
        
        # Summary for this file
        total_bound = len(l4_bound) + len(l5_bound)
        if total_bound > 0:
            print(f'\n  ✓ BINARIES FOUND: {total_bound} total ({len(l4_bound)} in L4, {len(l5_bound)} in L5)')
        else:
            print(f'\n  ✗ No bound binaries found')
        
        results_summary.append({
            'filename': filename,
            'total_particles': len(final_planetesimals),
            'l4_trojans': len(l4_idx),
            'l5_trojans': len(l5_idx),
            'l4_binaries': len(l4_bound),
            'l5_binaries': len(l5_bound),
            'l4_binaries_all': len(l4_binaries),
            'l5_binaries_all': len(l5_binaries),
            'status': 'Success'
        })
        
    except Exception as e:
        print(f'  ❌ Error: {str(e)}')
        results_summary.append({
            'filename': filename,
            'status': f'Error: {str(e)}'
        })

print(f'\n\n{"="*80}')
print('SUMMARY TABLE')
print(f'{"="*80}\n')
print(f"{'Filename':<45} {'Particles':>10} {'L4':>6} {'L5':>6} {'L4 Bin':>8} {'L5 Bin':>8}")
print('-'*80)

for result in results_summary:
    if result['status'] == 'Success':
        fname = result['filename'][:43] + '..' if len(result['filename']) > 45 else result['filename']
        print(f"{fname:<45} {result['total_particles']:>10} "
              f"{result['l4_trojans']:>6} {result['l5_trojans']:>6} "
              f"{result['l4_binaries']:>8} {result['l5_binaries']:>8}")
    else:
        fname = result['filename'][:43] + '..' if len(result['filename']) > 45 else result['filename']
        print(f"{fname:<45} {result['status']}")

# Count files with binaries
files_with_binaries = sum(1 for r in results_summary 
                         if r['status'] == 'Success' and 
                         (r['l4_binaries'] > 0 or r['l5_binaries'] > 0))

print('-'*80)
print(f'\nFiles with bound binaries: {files_with_binaries}/{len(pkl_files)}')
print('='*80)

BINARY SEARCH ACROSS ALL SIMULATION FILES

Found 11 .pkl files:


Analyzing: jupiter_burn_100kyr_5.2au_pure_disk.pkl
  Time range: 0.0 - 1000000.1 yr
  Final planetesimals: 10000
  L4 Trojans: 13
  L5 Trojans: 16
  Searching L4 for binaries...
    Close pairs: 0, Bound: 0
  Searching L5 for binaries...
    Close pairs: 0, Bound: 0

  ✗ No bound binaries found

Analyzing: jupiter_burn_5kyr_4.8au.pkl
  Time range: 0.0 - 5000.0 yr
  Final planetesimals: 10000
  L4 Trojans: 161
  L5 Trojans: 122
  Searching L4 for binaries...
    Close pairs: 0, Bound: 0
  Searching L5 for binaries...
    Close pairs: 0, Bound: 0

  ✗ No bound binaries found

Analyzing: jupiter_burn_5kyr_5.2au.pkl
  Time range: 0.0 - 5000.0 yr
  Final planetesimals: 10000
  L4 Trojans: 152
  L5 Trojans: 165
  Searching L4 for binaries...
    Close pairs: 0, Bound: 0
  Searching L5 for binaries...
    Close pairs: 0, Bound: 0

  ✗ No bound binaries found

Analyzing: jupiter_burn_5kyr_5.6au.pkl
  Time range: 0.0 - 20000.1 yr